In [1]:
using AutoGrad
using Deeplearning
using Statistics
using StatsBase
using Plots
import CSV

┌ Info: Recompiling stale cache file /Users/mertceylan/.julia/compiled/v1.1/Deeplearning/gtjFf.ji for Deeplearning [0d5bc7a0-39dd-11e9-2aab-59c8d4b467de]
└ @ Base loading.jl:1184
┌ Warning: Package Deeplearning does not have LinearAlgebra in its dependencies:
│ - If you have Deeplearning checked out for development and have
│   added LinearAlgebra as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Deeplearning
└ Loading LinearAlgebra into Deeplearning from project dependency, future warnings for Deeplearning are suppressed.


In [2]:
batch = 100
learning_rate = 10e-3

0.01

In [3]:
# test = CSV.read("datasets/digit-recognizer/test.csv")

In [4]:
train = CSV.read("datasets/digit-recognizer/train.csv")
xtrain = []
ytrain = []
for i in 1:size(train)[1]
    max = maximum(train[i,2:end])+0.1
    normalized = reshape(train[i, 2:end]./(max), (28,28,1,1))
    push!(xtrain, normalized)
    push!(ytrain, train[i,1]+1)
end
println("Ready...")

Ready...


In [5]:
input = @cudaarray rand(28, 28, 1, 10)
w1 = ((rand(3,3,1,6).-0.5)./100)
w2 = ((rand(3,3,6,16).-0.5)./100)
println("Ready...")

Ready...


In [6]:
conv1 = convolve(input, w1, (2,2), (1,1))
x = conv1(input)
# pool1 = avgpooling(x, (2,2), (2,2), (1,1))
# x = pool1(x)
conv2 = convolve(x, w2, (2,2), (1,1))
x = conv2(x)
# pool2 = avgpooling(x, (2,2), (2,2), (1,1))
# x = pool2(x)
dense1 = dense(x, 120)
x = dense1(x)
dense2 = dense(x, 84)
x = dense2(x)
dense3 = dense(x, 10)
layers=[dense3,dense2,dense1,conv2,conv1]
println("Loaded")

Loaded


In [7]:
function f(x)
    x = conv1(x)
    x = relu(x)
#     x = pool1(x)
    
    x = conv2(x)
    x = relu(x)
#     x = pool2(x)
    
    x = dense1(x)
    x = relu(x)

    x = dense2(x)
    x = relu(x)

    x = dense3(x)
    x = softmax(x)
end

f (generic function with 1 method)

In [8]:
acc_train_plot = []
acc_test_plot = []
loss_train_plot = []
loss_test_plot = []

0-element Array{Any,1}

In [ ]:
dff(x,y) = @diff nll(f(x), y)

for e in 1:2
    println("Epoch ", e)
    indices = sample(1:size(train)[1], size(train)[1], replace = false)
    for (ei,si) in enumerate(1:batch:size(indices)[1])
        sample_batch_x, sample_batch_y = cat(xtrain[indices[(si):(ei)*batch]]..., dims=4), ytrain[indices[(si):(ei)*batch]]

        dv = dff(sample_batch_x, sample_batch_y)

        dparams = @parameters dv

        for pidx in 1:length(dparams[1:end-1])
            dw = grad(dv, dparams[pidx])
            layers[pidx].matrix .-=learning_rate*dw
        end
    end
    otrain = cat(f.(xtrain), dims=2)
#     otest = f(xtest)
    push!(acc_train_plot, acc(otrain, ytrain))
    push!(loss_train_plot, nll(otrain, ytrain))
#     push!(acc_test_plot, acc(o, ytest))
#     push!(loss_test_plot, nll(o, ytest))
end

In [ ]:
plot([acc_train_plot, acc_test_plot], linewidth=2, title="Accuracy")

In [ ]:
plot([loss_train_plot, loss_test_plot], linewidth=2, title="Loss")

In [ ]:
loss_train_plot

In [ ]:
f(xtrain[2])